In [14]:
midi_notes = []
for note in range(128):
    midi_notes.append(440*pow(2, (note-69)/12))

def noteToHz (notes, bpm=120) :
    let2num = {'c': 0, 'c#': 1, 'db': 1, 'd': 2, 'd#': 3, 'eb': 3, 'e': 4, 'fb': 4, 'f': 5, 'e#': 5, 'F#': 6, 'gb': 6,
               'g': 7, 'g#': 8, 'ab': 8, 'a': 9, 'a#': 10, 'bb': 10, 'b': 11, 'cb': 11} #all possible note inputs
    whole_note = 240 / bpm
    output = []
    for chord in notes:
        #handling the note to frequency conversion first
        if type(chord[0]) == list: #this means it's a chord
            pitch = []
            for note in range(len(chord[0])):
                octave = int(chord[0][note][0]) * 12 + 12
                letter = let2num[chord[0][note][1:]]
                pitch.append(midi_notes[octave+letter])
        else: #it's a single note
            if chord[0] != 'rest':
                octave = int(chord[0][0]) * 12 + 12
                letter = let2num[chord[0][1:]]
                pitch = [midi_notes[octave+letter]]
            else: #it's a rest
                pitch = [0.0]
        
        #handling the length to second conversion second
        dur = chord[1]*whole_note
        output.append([pitch, dur])
    return output
        

example_notes = [[['3b', '3g', '4d'], 0.25], ['3a', 0.25], ['3g', 0.25], ['3a', 0.25],
                 ['3b', 0.25], ['3b', 0.25], ['3b', 0.25], ['rest', 0.25]]

out = noteToHz(example_notes)
print(out)

[[[246.94165062806206, 195.99771799087463, 293.6647679174076], 0.5], [[220.0], 0.5], [[195.99771799087463], 0.5], [[220.0], 0.5], [[246.94165062806206], 0.5], [[246.94165062806206], 0.5], [[246.94165062806206], 0.5], [[0.0], 0.5]]
